# Numerical analysis - Laboratory 11
## Numerical Integration

### Scope of this laboratory
We will implement the algorithms for estimating the numerical values of definite integrals.

## Quadrature formulas

Let us consider the continuous function $f:[a,b] \rightarrow \mathbb{R}$. We want to determine a sequence of coefficients, the weights $(a_i)_{i=\overline{1,n}}$ that correspond to nodes $x_i$ in the interval $[a,b]$ so as to approximate the integral of the function by a formula of the type:
$$\int_a^b f(x) dx \approxeq \sum_{i=1}^n a_i f(x_i)$$

This approximation is called the "quadrature formula". There are several ways we can establish such a formula:

- Quadrature **Newton - Cotes**: if we take equally spaced nodes such that $x_2 - x_1 = x_3 - x_2 = ... = x_n - x_{n-1}$
- **Cebîşev** quadrature: when all weights $a_i$ are equal

In Python, we can use the built-in function `quadrature` from `scipy.integrate`.

### Exercise
- The `my_fun` function is defined below. Calculate the definite integral from $-1$ to $1$ of this function using the built-in `quadrature` function. Test for different values of the constants $a,b$ and $c$. Consult the documentation to understand how to use the function and its output.

In [ ]:
import numpy as np
from scipy.integrate import quadrature

def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

result, error = quadrature(my_fun, a=0, b=1, args=(1, 2, 3))
print("Integral:", result)
print("Error estimate:", error)

## Formule de tip Newton - Cotes

We will further implement different numerical integration algorithms that are based on Newton-type formulas. There are several methods by choosing differently the coefficients $a_i$ and the nodes $x_i$:
- **Midpoint rule**: the area under the graph of the function is approximated by means of several rectangles of equal width but different height given by the value of the function $f$ at the middle of the rectangle, i.e. $f(\frac{x_i + x_{i+1}}{2}) $
- **Trapezoidal rule**: on the same idea, only trapezoids are used instead of rectangles, and the upper ends are given by $f(x_i)$ and $f(x_{i+1})$
- **Simpson's rule**: this method approximates the graph by means of several parabolas, for this reason it is also called the method of parabolas

### Midpoint rule
From the course, we have that the integral can be approximated by the sum:
$$\frac{b-a}{n} \sum_{i=0}^{n-1} f(\frac{x_i+x_{i+1}}{2})$$
with the error given by
$$\frac{(b-a)^2}{4n} ||f'||_\infty$$

### Exercise
Implement the midpoint rule to approximate the `my_fun` function. Compare the result with the output of the `quadrature` function for different values of $n$.

In [ ]:
import numpy as np
from scipy.integrate import quadrature

# Define the function to integrate
def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

# Midpoint Rule Implementation
def midpoint_rule(f, a, b, n, args=()):
    h = (b - a) / n
    total = 0
    for i in range(n):
        midpoint = a + h * (i + 0.5)
        total += f(midpoint, *args)
    return h * total

# Compare with quadrature method for different n
def compare_methods(a_val, b_val, c_val, interval_start=0, interval_end=1, n_vals=[2, 4, 8, 16, 32]):
    print(f"{'n':>3} | {'Midpoint':>10} | {'Quadrature':>12} | {'Abs Error':>10}")
    print("-" * 44)
    
    for n in n_vals:
        midpoint_result = midpoint_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        quad_result, _ = quadrature(my_fun, interval_start, interval_end, args=(a_val, b_val, c_val))
        error = abs(midpoint_result - quad_result)
        print(f"{n:>3} | {midpoint_result:>10.6f} | {quad_result:>12.6f} | {error:>10.2e}")

# Example usage
compare_methods(a_val=1, b_val=2, c_val=3)


### Trapezoidal rule
In this case, we have
$$ \frac{b-a}{n} \sum_{i=0}^{n-1} \frac{f(x_i)+f(x_{i+1})}{2}$$
and the error is given by
$$\frac{(b-a)^3}{12n^2}||f''||_\infty$$

### Exercise
- Implement this method as well and estimate the error in the approximation of the function `my_fun`.

In [ ]:
import numpy as np
from scipy.integrate import quadrature
from scipy.misc import derivative

# Define the function to integrate
def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

# Trapezoidal Rule Implementation
def trapezoidal_rule(f, a, b, n, args=()):
    x = np.linspace(a, b, n + 1)
    h = (b - a) / n
    total = 0.5 * f(x[0], *args) + 0.5 * f(x[-1], *args)
    for i in range(1, n):
        total += f(x[i], *args)
    return h * total

# Estimate error using the theoretical error bound
def estimate_trapezoidal_error(f, a, b, n, args=()):
    # Define a function to compute the second derivative
    def f_second_derivative(x):
        return derivative(lambda x_: derivative(lambda x__: f(x__, *args), x_, dx=1e-6), x, dx=1e-6)
    
    # Estimate ||f''||_∞ by evaluating at many points
    sample_points = np.linspace(a, b, 1000)
    second_derivs = np.abs([f_second_derivative(x) for x in sample_points])
    max_f_double_prime = np.max(second_derivs)
    
    error_bound = ((b - a) ** 3 / (12 * n ** 2)) * max_f_double_prime
    return error_bound

# Compare method and error estimation
def compare_trapezoidal(a_val, b_val, c_val, interval_start=0, interval_end=1, n_vals=[2, 4, 8, 16, 32]):
    print(f"{'n':>3} | {'Trapezoid':>10} | {'Quadrature':>12} | {'Abs Error':>10} | {'Est. Error':>10}")
    print("-" * 60)
    
    for n in n_vals:
        trap_result = trapezoidal_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        quad_result, _ = quadrature(my_fun, interval_start, interval_end, args=(a_val, b_val, c_val))
        abs_error = abs(trap_result - quad_result)
        est_error = estimate_trapezoidal_error(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        print(f"{n:>3} | {trap_result:>10.6f} | {quad_result:>12.6f} | {abs_error:>10.2e} | {est_error:>10.2e}")

# Example usage
compare_trapezoidal(a_val=1, b_val=2, c_val=3)


- Also implement the composite variant of the trapezoidal rule
$$ \frac{b-a}{n}[ \frac{f(a) + f(b)}{2} + \sum_{i=1}^{n-1} f(x_i)]$$
The formula is equivalent to the one given above, except that implementing it in this form makes the algorithm faster. Check thi by comparing the execution times for estimating integrals over larger intervals (if the times are too small, run the estimate several times, using a `for` loop, and measure the total time. Increase the number of iterations in the loop until when you notice a difference between the simple and composite version).

In [ ]:
import numpy as np
from scipy.integrate import quadrature
import time

# Define the function to integrate
def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

# Basic loop-based Trapezoidal Rule
def trapezoidal_rule(f, a, b, n, args=()):
    x = np.linspace(a, b, n + 1)
    h = (b - a) / n
    total = 0.5 * f(x[0], *args) + 0.5 * f(x[-1], *args)
    for i in range(1, n):
        total += f(x[i], *args)
    return h * total

# Composite (vectorized) Trapezoidal Rule
def composite_trapezoidal_rule(f, a, b, n, args=()):
    x = np.linspace(a, b, n + 1)
    fx = f(x, *args) if callable(getattr(f, '__call__', None)) else np.vectorize(f)(x, *args)
    h = (b - a) / n
    return h * (0.5 * fx[0] + 0.5 * fx[-1] + np.sum(fx[1:-1]))

# Timing comparison
def timing_comparison(a_val, b_val, c_val, interval_start=0, interval_end=10, n=1000, repetitions=1000):
    print("Running timing comparison...")
    
    # Time basic trapezoidal rule
    start_time = time.time()
    for _ in range(repetitions):
        trapezoidal_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
    basic_time = time.time() - start_time

    # Time composite version
    start_time = time.time()
    for _ in range(repetitions):
        composite_trapezoidal_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
    composite_time = time.time() - start_time

    print(f"Basic Trapezoidal Rule Time     : {basic_time:.4f} seconds")
    print(f"Composite Trapezoidal Rule Time : {composite_time:.4f} seconds")
    print(f"Speed-up: {basic_time / composite_time:.2f}x")

# Run comparison
timing_comparison(a_val=1, b_val=2, c_val=3, interval_start=0, interval_end=10, n=1000, repetitions=1000)


### The Simpson method
This method has the lowest error of the three Newton-Cotes quadratures studied in this laboratory. The implemented formula will be
$$\frac{b-a}{6n} \sum_{i=0}^{n-1} [f(x_i) + 4f(\frac{x_i+x_{i+1}}{2}) + f( x_{i+1})]$$

### Exercise
Also implement the Simpson method. Plot on the same graph for the three methods the absolute error (compare with the output of the function `quadrature`) according to the number of considered divisions, $n$.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import quadrature

# Define the function
def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

# Midpoint Rule
def midpoint_rule(f, a, b, n, args=()):
    h = (b - a) / n
    result = 0
    for i in range(n):
        xi = a + i * h
        xi1 = xi + h
        result += f((xi + xi1) / 2, *args)
    return h * result

# Trapezoidal Rule
def trapezoidal_rule(f, a, b, n, args=()):
    h = (b - a) / n
    result = 0.5 * f(a, *args) + 0.5 * f(b, *args)
    for i in range(1, n):
        result += f(a + i * h, *args)
    return h * result

# Simpson's Rule
def simpson_rule(f, a, b, n, args=()):
    h = (b - a) / n
    result = 0
    for i in range(n):
        xi = a + i * h
        xi1 = xi + h
        midpoint = (xi + xi1) / 2
        result += f(xi, *args) + 4 * f(midpoint, *args) + f(xi1, *args)
    return h * result / 6

# Plot errors for all three methods
def plot_errors(a_val, b_val, c_val, interval_start=0, interval_end=1):
    n_values = [2**i for i in range(1, 9)]  # n = 2, 4, 8, ..., 256
    midpoint_errors = []
    trapezoid_errors = []
    simpson_errors = []

    # Compute reference "true" value from quadrature
    true_val, _ = quadrature(my_fun, interval_start, interval_end, args=(a_val, b_val, c_val))

    for n in n_values:
        midpoint = midpoint_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        trapezoid = trapezoidal_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        simpson = simpson_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))

        midpoint_errors.append(abs(midpoint - true_val))
        trapezoid_errors.append(abs(trapezoid - true_val))
        simpson_errors.append(abs(simpson - true_val))

    # Plot
    plt.figure(figsize=(10, 6))
    plt.loglog(n_values, midpoint_errors, 'o-', label='Midpoint Rule')
    plt.loglog(n_values, trapezoid_errors, 's-', label='Trapezoidal Rule')
    plt.loglog(n_values, simpson_errors, '^-', label="Simpson's Rule")

    plt.xlabel('Number of intervals (n)')
    plt.ylabel('Absolute Error')
    plt.title('Error Comparison of Newton-Cotes Methods')
    plt.legend()
    plt.grid(True, which='both', linestyle='--', alpha=0.6)
    plt.tight_layout()
    plt.show()

# Run the plot function
plot_errors(a_val=1, b_val=2, c_val=3)


## Gaussian quadrature
If we define the weighting function $w:I\rightarrow \mathbb{R}$ on the integration interval so that the equation
$$\int_a^b w(x)f(x) dx = \sum_{i=1}^n a_i f(x_i)$$
to be satisfied, then we have the following classical methods:

<img width=600 src="https://github.com/prodangp/LaboratorCN/blob/main/media/lab11/quadratures.png?raw=true"/>

These methods can be quickly implemented with `numpy`, for example using the `leggauss` function ([docs](https://numpy.org/doc/stable/reference/generated/numpy.polynomial.legendre.leggauss.html)) :

In [ ]:
import numpy as np
from scipy.integrate import quadrature

# Define your function
def my_fun(x, a, b, c):
    return (a + b * np.sin(x)) / (4 + c * np.exp(x ** 2))

# Gauss-Legendre quadrature on [a, b]
def gauss_legendre_rule(f, a, b, n, args=()):
    # Get nodes and weights for interval [-1, 1]
    nodes, weights = np.polynomial.legendre.leggauss(n)

    # Change of variables to [a, b]
    mapped_nodes = 0.5 * (nodes + 1) * (b - a) + a
    mapped_weights = 0.5 * (b - a) * weights

    # Evaluate function at mapped nodes
    values = f(mapped_nodes, *args)
    return np.sum(mapped_weights * values)

# Compare Gauss–Legendre with scipy quadrature
def compare_gaussian(a_val, b_val, c_val, interval_start=0, interval_end=1, n_vals=[2, 4, 8, 16]):
    true_val, _ = quadrature(my_fun, interval_start, interval_end, args=(a_val, b_val, c_val))
    
    print(f"{'n':>3} | {'Gauss-Legendre':>15} | {'True':>10} | {'Abs Error':>10}")
    print("-" * 50)
    
    for n in n_vals:
        approx = gauss_legendre_rule(my_fun, interval_start, interval_end, n, args=(a_val, b_val, c_val))
        error = abs(approx - true_val)
        print(f"{n:>3} | {approx:>15.8f} | {true_val:>10.8f} | {error:>10.2e}")

# Example run
compare_gaussian(a_val=1, b_val=1, c_val=3)

import numpy as np

# Example functions
f_chebyshev = lambda x: x**2        # Works well with Chebyshev weight
f_laguerre  = lambda x: x**2        # Works well with Laguerre weight
f_hermite   = lambda x: x**2        # Works well with Hermite weight

# Gauss–Chebyshev
def gauss_chebyshev(f, n):
    i = np.arange(1, n + 1)
    x_i = np.cos((2 * i - 1) * np.pi / (2 * n))
    weights = np.pi / n
    return weights * np.sum(f(x_i))

# Gauss–Laguerre
def gauss_laguerre(f, n):
    x, w = np.polynomial.laguerre.laggauss(n)
    return np.sum(w * f(x))

# Gauss–Hermite
def gauss_hermite(f, n):
    x, w = np.polynomial.hermite.hermgauss(n)
    return np.sum(w * f(x))

# Run all
def run_all(n=10):
    print(f"Using n = {n} nodes\n")

    I_cheb = gauss_chebyshev(f_chebyshev, n)
    print(f"Gauss–Chebyshev (∫ x² / √(1-x²) dx on [-1,1]):  {I_cheb:.6f}")

    I_lag  = gauss_laguerre(f_laguerre, n)
    print(f"Gauss–Laguerre  (∫ x² e^(-x) dx on [0, ∞)):     {I_lag:.6f} (expected 2)")

    I_herm = gauss_hermite(f_hermite, n)
    print(f"Gauss–Hermite   (∫ x² e^(-x²) dx on ℝ):         {I_herm:.6f} (expected √π/2 ≈ {np.sqrt(np.pi)/2:.6f})")

# Run demo
run_all(n=10)



### Exercise
Approximate the same integral using Gauss - Chebyshev (see [documentation](https://numpy.org/doc/stable/reference/routines.polynomials.chebyshev.html)). Check the result obtained by comparing the result of the same approximation by the previously implemented methods (Newton - Cotes type).

### Multiple integrals

A double integral defined on $D=\{(x,y) | x \in [a,b], \ y \in [c,d]\}$ can be approximated with the built-in function `dblquad`, and for triple integrals we can use `tplquad`. Both are found in the `integrate` module of `scipy`.

In [ ]:
from scipy import integrate

# Function f(y, x) = x * y^2
f = lambda y, x: x * y ** 2

# Integrate over x ∈ [0, 2], y ∈ [0, 1]
result, error = integrate.dblquad(f, 0, 2, lambda x: 0, lambda x: 1)

print(f"Double integral result: {result:.6f}")
print(f"Estimated error: {error:.2e}")


### Exercise
Consider the function $f:[0,1] \times [0,1] \rightarrow \mathbb{R}$ such that $f(x,y)=e^{-3y}\sin (x^2) + \ cos(xy)$. Test `dblquad` to integrate the function over the given interval. The result should be $-0.221766...$

In [ ]:
from scipy import integrate
import numpy as np

# Define the function: note order (y, x) for dblquad
f = lambda y, x: np.exp(-3 * y) * np.sin(x ** 2) + np.cos(x * y)

# Integration limits for x in [0, 1], and y in [0, 1] for every x
result, error = integrate.dblquad(f, 0, 1, lambda x: 0, lambda x: 1)

# Display result
print(f"Double integral result: {result:.6f}")
print(f"Estimated error: {error:.2e}")
